<a href="https://colab.research.google.com/github/shashi29/Text-code-repo/blob/master/ImageProcessing_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import sys
import glob

import matplotlib.pyplot as plt
import csv
try:
    from PIL import Image
except ImportError:
    import Image

from google.colab.patches import cv2_imshow

## Basic Image processing :
Extract all the rectangles

In [34]:
def sort_contours(cnts, method="left-to-right"):
  # initialize the reverse flag and sort index
  reverse = False
  i = 0
  # handle if we need to sort in reverse
  if method == "right-to-left" or method == "bottom-to-top":
      reverse = True
  # handle if we are sorting against the y-coordinate rather than
  # the x-coordinate of the bounding box
  if method == "top-to-bottom" or method == "bottom-to-top":
      i = 1
  # construct the list of bounding boxes and sort them from top to
  # bottom
  boundingBoxes = [cv2.boundingRect(c) for c in cnts]
  (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
  key=lambda b:b[1][i], reverse=reverse))
  # return the list of sorted contours and bounding boxes
  return (cnts, boundingBoxes)

def find_vertical_horizontal(img, img_bin):

  kernel_len = np.array(img).shape[1]//100
  ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
  hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
  image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
  vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=6)
  
  image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
  horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=25)
  horizontal_lines = cv2.erode(horizontal_lines,hor_kernel,iterations = 20)
  
  img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
  img_vh = cv2.erode(~img_vh, kernel, iterations=2)
  thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  return img_vh

def process(file):
  #Step 1
  imgOrig = cv2.imread(file)
  img = cv2.imread(file,0)
  thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY |cv2.THRESH_OTSU)
  #inverting the image 
  img_bin = 255-img_bin

  #Step 2
  table_img = find_vertical_horizontal(img, img_bin)
  #cv2_imshow(table_img)

  #Step 3
  contours, hierarchy = cv2.findContours(table_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort all the contours by top to bottom.
  contours, boundingBoxes = sort_contours(contours, method="top-to-bottom")

  #Creating a list of heights for all detected boxes
  heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]
  #Get mean of heights
  mean = np.mean(heights)

  box = []
  crop_img = []
  # Get position (x,y), width and height for every contour and show the contour on image
  for c in contours:
      x, y, w, h = cv2.boundingRect(c)
      if (h>10):
          image = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
          box.append([x,y,w,h])
          crop_im = imgOrig[y:y+h,x:x+w]
          #cv2.namedWindow("crop image",cv2.WINDOW_NORMAL)
          #cv2_imshow(crop_im)
          crop_img.append(crop_im)
          #cv2.waitKey(0)    
  #cv2_imshow(image)
  return crop_img, box


In [3]:
#img_path = "/content/hook.jpg"
#for img_path in glob.glob("*.JPEG"):
#crop_imgs, box = process(img_path)

##Text detctor and reconization network

In [4]:
!git clone https://github.com/dipu-bd/craft-moran-ocr.git
!mv craft-moran-ocr/* .

Cloning into 'craft-moran-ocr'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 77 (delta 22), reused 63 (delta 12), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [5]:
!bash init.sh

mkdir: created directory 'data'

--------- DOWNLOADING TRAINED DATA FOR CRAFT ------------
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    155      0 --:--:--  0:00:02 --:--:--   155
100 79.3M    0 79.3M    0     0  19.8M      0 --:--:--  0:00:03 --:--:-- 61.5M
craft_mlt_25k.pth | 2f8227d2def4037cdb3b34389dcf9ec1 | verified

---------- DOWNLOADING REFINER DATA FOR CRAFT -----------
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    686      0 --:--:-- --:--:-- --:--:--   685
100 1810k  100 1810k    0     0  2218k      0 --:--:-- --:--:-- --:--:-- 2218k
craft_refiner_CTW1500.pth | 3d48f83540567d2a43d2f6ca4b4d9f13 | verified

---------- DOWNLOADING TRAINED DATA FOR MORAN -----------
  % Total    % Received

In [6]:
from src.recognizer import Recognizer
from src.detector import Detector

#Load the model for Text detection and recognization
detector = Detector()
detector.load()
recognizer = Recognizer()
recognizer.load()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Loading pretrained model from /content/data/moran_v2_demo.pth


In [7]:
!pip install colour
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 33 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,196 kB/s)
debconf: unable to initi

In [8]:
import pytesseract
import warnings
warnings.filterwarnings('ignore')

In [9]:

def text_detection_recognition(crop_imgs, box):
  info = list()
  for img, bb in zip(crop_imgs, box):
      ocr_flag = 0
      h, w,_ = img.shape
      #cv2_imshow(img)
      crop_imgs,_,_,_ = detector.process(img)
      result = ''
      for im in crop_imgs:
          try:
            #cv2_imshow(im)
            gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            text, _, _ = recognizer.process(gray)
            result += text
            result += ' '
          except Exception as ex:
            print(ex)
      flag = 1
      #print(result)
      if flag == 1:
        extractedInformation = pytesseract.image_to_string(img)
        crop_imgs,_,_,_ = detector.process(img)
        if len(crop_imgs):
          info.append([result,bb,extractedInformation])
          #print(result,bb,h*w)
  return info

#img_path = "/content/Ashok-10002972.pdfpage_1.JPEG"
#crop_imgs, box = process(img_path)
#info = text_detection_recognition(crop_imgs, box)

##Filter out Tables from that

In [10]:
####invoice processing 05-07-2020
import os
import numpy as np
import pandas as pd
from scipy import stats
from collections import OrderedDict
import statistics
#filename="datapoint.txt"
#
#with open(filename) as fp:
#    lines=fp.readlines()
    
    
#datapoint=[['slno 0 ', [202, 1289, 336, 127], 'SL.NO.'], ['item descriptions ', [548, 1290, 1587, 142], 'ITEM DESCRIPTIONS'], ['hsn code ', [2146, 1309, 541, 130], 'HSN CODE'], ['qtykg ', [2698, 1315, 409, 129], 'QTY./Kg.'], ['rate ', [3118, 1321, 454, 128], 'RATE\ni Ra aig'], ['amount ', [3582, 1326, 416, 126], 'AMOUNT'], ['baby corn ', [547, 1427, 1587, 93], 'BABY CORN'], ['10059000 ', [2146, 1445, 539, 82], '10059000 —'], ['050 ', [2698, 1451, 408, 81], '0.50'], ['7000 ', [3118, 1457, 452, 80], '70.00'], ['3500 ', [3582, 1462, 415, 77], '35.00'], ['2 ', [199, 1514, 336, 74], ''], ['brocolli wo steam ', [546, 1515, 1586, 91], 'BROCOLLI W/O STEAM\n\nCe\n\na'], ['300 ', [2697, 1539, 407, 78], ''], ['12000 ', [3117, 1544, 450, 78], '120.00'], ['360 00 ', [3580, 1550, 416, 74], '360.00'], ['3 ', [198, 1599, 335, 75], ''], ['capsicum red ', [546, 1601, 1586, 91], 'CAPSICUM RED'], ['07096090 ', [2144, 1619, 540, 80], '07096090'], ['010 ', [2696, 1626, 407, 78], 'aS'], ['11000 ', [3115, 1630, 451, 78], ''], ['11 00 ', [3579, 1636, 416, 75], '11.00'], ['4 ', [197, 1685, 336, 75], ''], ['capsicum yellow ', [544, 1687, 1587, 91], 'CAPSICUM YELLOW'], ['07096090 ', [2142, 1705, 540, 80], '07096090'], ['010 ', [2695, 1712, 407, 77], ''], ['11000 ', [3114, 1717, 451, 78], '110.00'], ['11 00 ', [3576, 1722, 417, 75], '11.00'], ['51 ', [196, 1771, 337, 76], ''], ['lettuce iceburg ', [544, 1773, 1586, 92], 'LETTUCE ICEBURG'], ['07051900 ', [2142, 1791, 540, 81], '07051900'], ['050 ', [2694, 1798, 407, 78], '0.50 —'], ['8000 ', [3113, 1803, 450, 79], '~ 80.00'], ['4000 ', [3575, 1808, 415, 76], '40.00'], ['lettuce green ', [544, 1859, 1585, 92], 'LETTUCE GREEN'], ['07051900 ', [2140, 1877, 540, 81], '07051900'], ['040 ', [2692, 1885, 408, 79], '0.40 —'], ['9000 ', [3112, 1890, 448, 78], '90.00'], ['361 00 ', [3573, 1895, 415, 76], '36.00'], ['lettuce lolorosa ', [543, 1946, 1585, 93], 'LETTUCE LOLOROSA'], ['07051900 ', [2139, 1965, 540, 81], '07051900'], ['040 ', [2691, 1971, 409, 80], '0.40'], ['100 00 ', [3111, 1976, 447, 79], '100.00'], ['4000 ', [3571, 1982, 415, 76], '40.00'], ['mashroom fresh ', [542, 2033, 1585, 94], 'MASHROOM FRESH'], ['07095100 ', [2138, 2051, 540, 83], '07095100'], ['2 a 00 ', [2690, 2058, 409, 81], '2.00'], ['11000 ', [3110, 2064, 447, 80], 'eee yy'], ['22000 ', [3568, 2069, 416, 77], '220.00'], ['red radish ', [541, 2121, 1585, 93], 'RED RADISH'], ['07069020 ', [2137, 2139, 540, 82], '07069020'], ['020 ', [2689, 2146, 410, 80], '0.20'], ['401 00 ', [3109, 2151, 448, 80], '~ 40.00.'], ['800 ', [3568, 2157, 416, 76], ''], ['10 ', [189, 2207, 339, 77], ''], ['beans pencil ', [539, 2209, 1587, 93], 'BEANS PENCIL rere'], ['07082000 ', [2136, 2227, 541, 83], '07082000'], ['200 ', [2688, 2234, 411, 81], ''], ['3700 ', [3109, 2240, 448, 80], '37.00'], ['7400 ', [3568, 2245, 415, 77], ''], ['rimarks total qty 9 20 total amt a 8351 00 ', [187, 2307, 3797, 203], 'Rrmarks_: y Total Qty.: 9.20 Total Amt: 835.00\npte alae lene a a'], ['re ', [3751, 2865, 229, 117], '']]

   
#datapoint=[['slno ', [250, 1249, 341, 129], 'SL.NO.'], ['item descriptions ', [601, 1253, 1590, 159], 'â€œAst man raroaeonrnrerereex._ â€œi _\n\nITEM DESCRIPTIONS'], ['hsn code ', [2201, 1287, 542, 136], 'HSN CODE'], ['qtykg ', [2754, 1299, 410, 133], 'QTY /Kg.'], ['rate ', [3176, 1309, 450, 132], 'RATE'], ['amount ', [3636, 1318, 418, 126], 'AMOUNT'], ['baby corn ', [601, 1391, 1588, 109], 'nite raAaAn+ 2) h (Sh rrsS Fk â€” rr\n_ BABY CORN _'], ['10059000 ', [2200, 1425, 541, 87], '10059000'], ['050 ', [2753, 1437, 410, 84], '0.50'], ['70 00 ', [3175, 1446, 450, 84], '70.00'], ['35 00 ', [3636, 1455, 414, 77], '35.00'], ['brocolli wo steam wiv ', [599, 1480, 1588, 105], '; BROCOLLI W W/O STEAM'], ['07041000 ', [2199, 1513, 540, 83], '07041000'], ['the ', [2751, 1525, 410, 81], 'E\n\nUk'], ['12000 ', [3174, 1534, 449, 81], '120.00'], ['1200 00 ', [3636, 1544, 411, 73], '1200.00'], ['010 ', [2750, 1610, 410, 81], 'ae) ow'], ['10000 ', [3172, 1619, 448, 82], '-+00:00-'], ['capsicum red ', [596, 1650, 1587, 106], 'SCAPSICUMEED . = tâ€”t~â€”CSSS\nessences...'], ['07096090 ', [2195, 1684, 541, 84], '07096090 |'], ['010 ', [2749, 1696, 407, 81], '0.10'], ['capsicum yellow ', [595, 1737, 1587, 106], 'CAPSICUM YELLOW #22'], ['07096090 ', [2193, 1770, 542, 85], '07096090'], ['010 ', [2747, 1783, 407, 81], '0.10'], ['11 00 ', [3624, 1802, 418, 74], '11.00'], ['07094000 ', [2191, 1857, 541, 86], '07094000'], ['lettuce iceburg yovi ', [594, 1912, 1584, 106], '_LETTUCEICEBURG â€”(stâ€”=~wC a)'], ['07051900 ', [2190, 1945, 541, 86], '07051900'], ['050 ', [2744, 1958, 408, 82], '\\\\ oY q O56"'], ['8000 ', [3164, 1967, 447, 82], '80.00'], ['4000 ', [3623, 1977, 415, 75], '~ 40.00'], ['mashroom fresh ', [590, 2000, 1585, 107], '* MASHROOM FRESH\nESN'], ['07095100 ', [2188, 2033, 540, 87], '07095100'], ['300 ', [2741, 2045, 408, 84], '3.00'], ['11000 ', [3162, 2055, 448, 84], 'GH'], ['33000 ', [3621, 2065, 415, 76], '330.00'], ['07007070 ', [2186, 2156, 540, 51], 'VIVOFVULY'], ['10 ', [235, 2174, 337, 77], 'a'], ['snow peas imp ', [585, 2175, 1587, 109], 'PSNOWPEASIMP. ='], ['07099090 ', [2184, 2210, 541, 86], '07099090'], ['020 ', [2737, 2222, 410, 82], '0.20'], ['25000 ', [3158, 2231, 450, 83], 'â€” 250.00'], ['spring onion ', [583, 2265, 1588, 106], 'aNCONON.  .'], ['07039000 ', [2182, 2297, 541, 87], '07039000'], ['100 ', [2735, 2310, 411, 82], '1.00'], ['apple red delicious imp ', [580, 2353, 1589, 105], 'â€œ APP\n\nLE RED DELICIOUS IMP.'], ['08081000 ', [2181, 2385, 541, 85], '08081000'], ['1 00 ', [2734, 2397, 411, 82], '1.00'], ['16060 ', [3157, 2406, 448, 82], '160.50'], ['16000 ', [3617, 2416, 418, 77], '160.00'], ['orange imp ', [578, 2439, 1589, 105], '_ ORANGE IMP.'], ['08051000 ', [2179, 2471, 541, 85], '08051000'], ['3 00 ', [2732, 2484, 410, 81], 'eae Ki'], ['95 o 00 ', [3154, 2493, 451, 82], '95.00'], ['285 00 ', [3615, 2502, 417, 76], '285.00'], ['beans pencil ', [577, 2525, 1590, 107], '~ BEANS PENCIL'], ['07082000 ', [2177, 2557, 543, 88], '07082000'], ['2 o 00 ', [2730, 2570, 412, 83], '2.00'], ['7400 ', [3615, 2589, 415, 78], 'Lee'], ['rrmarks judy 1400 total qty 2190 total amt 2278 00 ', [226, 2623, 3802, 234], 'Rrmarks\n\nTotal Qty.: 21.90 Total Amt.: 2278.00'], ['calve ', [2920, 3197, 190, 109], 'CA WV.']]
def restructureDataPoints(datapoint):
    """ take list of list data point as an input"""
    crnnNetwork=[]
    ocrouput=[]
    x1cor,y1cor,height,width,x2cor,y2cor=[[] for i in range(6)]
    for dpoints in datapoint:
        try:
        #data=dpoints.split(",")
            crnnNetwork.append(dpoints[0])
            x1,y1,h,w=dpoints[1]
            ocrouput.append(dpoints[2])
            x1cor.append(x1)
            y1cor.append(y1)
            height.append(h)
            width.append(w)
            x2cor.append(x1+w)
            y2cor.append(y1+h)
        except Exception as e:
            print("exception caught",str(e),dpoints)
    
    datapointsfortables=pd.DataFrame()
    datapointsfortables['CrnnNetworkOutput']=crnnNetwork
    datapointsfortables['ocrOutput']=ocrouput
    datapointsfortables['x1cor']=x1cor
    datapointsfortables['x2cor']=x2cor
    datapointsfortables['y1cor']=y2cor
    datapointsfortables['y2cor']=y1cor
    datapointsfortables['height']=height
    datapointsfortables['width']=width
    return datapointsfortables
#datapointsfortables['x1cor']=x1cor
def sortDataFrame(df):
    sortedDatapoints=df.sort_values(by=['y2cor', 'x1cor'],ascending=[True,True]).reset_index(drop=True)
    return sortedDatapoints

##get the potential line breaker
'''
def getPotentialLineData(sortedDatapoints):
    z = np.abs(stats.zscore(sortedDatapoints['height']))
    print(z)
    #outlier threshold
    #threshold = 1
    indexs=np.where(z > 1)[0].tolist()
    linesData=[]
    
    i=indexs[0]
    for lbreak in range(1,len(indexs)):
        j=indexs[lbreak]
        tdf=sortedDatapoints.iloc[i:j,:] 
        linesData.append(tdf)
        i=j
    return linesData
'''
def getPotentialLineBreak(linedata):
    x1_cordinates=linedata['x1cor']
    linebreak=[]
    intialPOs=x1_cordinates[0]
    for idx in range(1,len(x1_cordinates)-1):
        cor=x1_cordinates[idx]
#        print("idx",idx)
#        print("intialPO",intialPOs)
#        print("cor",cor)
        if cor<intialPOs:
            linebreak.append(idx)
            intialPOs=x1_cordinates[idx+1]
        elif cor>=intialPOs:
            intialPOs=cor 
    return linebreak

def getLinebyLineData(sortedTablePoints,linebreak):
    linesData=[]
    i=0
    for lbreak in range(len(linebreak)):
        j=linebreak[lbreak]
        tdf=sortedTablePoints.iloc[i:j,:] 
        linesData.append(tdf)
        i=j
    return linesData

##find column data
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

#find_nearest(linesData[0]['x1cor'] ,3164)
def findDfwithMaxElements(linesData):
#find the df which have maximum no of elements
    lineWithlengthDict=OrderedDict() 
    for idx, df in enumerate(linesData):
        lineWithlengthDict[idx]=df.shape[0]
    
    Keymax = max(lineWithlengthDict, key=lineWithlengthDict.get)
    return Keymax

def getMediumLengthOfTable(linesData):
    dfShapes=[line.shape[0] for line in linesData]
    res=statistics.median(dfShapes)
    if res in dfShapes:
        return dfShapes.index(int(res))
    else:
        val=find_nearest(dfShapes,res)
        print(val)
        print(dfShapes)
        return dfShapes.index(val)

def constructTable(linesData,kymax):
    intislDf=linesData[kymax]
    area_dict = dict(zip(intislDf['x1cor'], intislDf['CrnnNetworkOutput']))
    finalDict={}
    finalDict['lineNo']=[0]
    for orgKy in area_dict:
        finalDict[orgKy]=[area_dict[orgKy]]
        
    #firstRowlength=len(finalDict.keys()) 
    linebyLineDict={} 
    #linebyLineDict[0]=area_dict
    
    #pageNumber={}
    for idx,dfs in enumerate(linesData):
        #pageNumber={}
        temp={}
        t_dict = dict(zip(dfs['x1cor'], dfs['CrnnNetworkOutput']))
        #tdictlength=len(t_dict.keys())
        for ky in t_dict.keys():
            nearest=find_nearest(list(area_dict.keys()),ky)
            temp[nearest]=t_dict[ky]
        #pageNumber[idx+1]=temp
        linebyLineDict[idx+1]=temp
        
    resultDataFrame1=pd.DataFrame.from_dict(linebyLineDict, orient='index')
    return resultDataFrame1

def getTable(datapoint):
    tablepoints=restructureDataPoints(datapoint)
    sortedTablePoints=sortDataFrame(tablepoints)
    linebreak=getPotentialLineBreak(sortedTablePoints)
    linebylinedata=getLinebyLineData(sortedTablePoints,linebreak)
    #linesData=getPotentialLineData(sortedTablePoints)
    kymax=getMediumLengthOfTable(linebylinedata)
    table=constructTable(linebylinedata,kymax)
    return table,linebylinedata

def postProcessTable(table):
    table['full_count'] = table.apply(lambda x: (x.count())/len(x), axis=1)
    table1=table[table['full_count']>0.3].reset_index(drop=True)
    del table1['full_count']
    return table1


In [11]:
#First pass get all the info about the table
#Second remove the bigger one
#Third now for the bigger again the same flow has to be pass into
#Then combine everything in the multiple list
def crop_text_detection(img):
  crop_imgs,boxes,_,_ = detector.process(img)
  rects = list()
  for box in boxes:
      poly = np.array(box).astype(np.int32)
      y0, x0 = np.min(poly, axis=0)
      y1, x1 = np.max(poly, axis=0)
      rects.append([x0, y0, x1, y1])

  full_text = ''
  result = list()
  for indx,rect in enumerate(rects):
    try:
      x0,y0,x1,y1 = rect
      crop_img = img[x0:x1, y0:y1]
      #crop_img = img[y0:y1, x0:x1, :]
      gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
      text, _, _ = recognizer.process(gray)
      ocr_text = pytesseract.image_to_string(crop_img)
      result.append([text,[y0,x0,y1-y0,x1-x0],ocr_text])
      full_text += text
      full_text += ' '
    except Exception as ex:
      print(ex)

  return result

##get the potential line breaker
def getOUtlier(sortedDatapoints):
    z = np.abs(stats.zscore(sortedDatapoints['height']))
    print(z)
    #outlier threshold
    #threshold = 1
    indexs=np.where(z > 1)[0].tolist()
    linesData=[]
    print("Info inde",indexs)
    outlier = np.where(z > 2)[0].tolist()
    i=indexs[0]
    for lbreak in range(1,len(indexs)):
        j=indexs[lbreak]
        tdf=sortedDatapoints.iloc[i:j,:] 
        linesData.append(tdf)
        i=j
    return linesData, outlier

In [41]:
img_path = "/content/Ashok-10002972.pdfpage_1.JPEG"
img = cv2.imread(img_path)
crnn_output = crop_text_detection(img)

In [42]:
print(crnn_output)

[['marka', [279, 82, 240, 100], 'Marka'], ['avc', [576, 103, 180, 77], 'AVC'], ['ashoka', [1261, 126, 573, 145], 'ASHOKA'], ['vegetable', [1880, 144, 822, 145], 'VEGETABLE'], ['centre', [2725, 171, 559, 121], ''], ['english', [1504, 292, 361, 86], ''], ['vegetable', [1881, 291, 493, 98], 'VEGETABLE'], ['suppliers', [2381, 309, 447, 86], ''], ['a1', [1427, 412, 129, 78], ''], ['166new', [1553, 408, 340, 100], '166,NEW'], ['subzi', [1908, 421, 250, 86], ''], ['mandi', [2171, 421, 310, 106], 'MANDI,'], ['azadpur', [2493, 438, 404, 86], ''], ['mob', [1006, 533, 206, 77], ''], ['9654472544', [1279, 526, 433, 102], '9654472544'], ['9212573680', [1708, 535, 444, 108], '59212573680,'], ['9999755503', [2147, 547, 434, 106], '9999755503'], ['ph', [2588, 567, 120, 77], ''], ['601', [2725, 567, 112, 86], ''], ['127682054', [2826, 563, 426, 102], '1-27682054'], ['bill', [266, 619, 146, 77], ''], ['noil', [421, 627, 129, 78], 'No.'], ['2858', [834, 636, 180, 86], '2858'], ['dated', [2863, 722, 215, 

In [35]:
img_path = "/content/Ashok-10002972.pdfpage_1.JPEG"
combine = list()
for img_path in glob.glob("/content/*.JPEG"):
  crop_imgs, box = process(img_path)
  info = text_detection_recognition(crop_imgs, box)

  tablepoints=restructureDataPoints(info)
  sortedTablePoints=sortDataFrame(tablepoints)
  #lineData,z_indexes =getOUtlier(sortedTablePoints)
  combine.append(sortedTablePoints)
  '''
  #Filtering
  final_info = list()
  indexes_to_remove = z_indexes
  other_list = list()
  img = cv2.imread(img_path)
  for indx,im in enumerate(info):
    if indx in indexes_to_remove:
      #Write the crop and pass the path
      bb = im[1]
      crop_img = img[bb[1]:bb[1]+bb[3],bb[0]:bb[0]+bb[2],:]
      new_info = crop_text_detection(crop_img)
      final_info.append(new_info)
    else:
      other_list.append(im)

  final_info.append(other_list)
  #for info in final_info:
  #  display(getTable(info))
  img_info = ({img_path:final_info})
  combine.append(img_info)
  print(final_info)
  '''

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



In [36]:
combine[0]

,CrnnNetworkOutput,ocrOutput,x1cor,x2cor,y1cor,y2cor,height,width
0,marka avc ashoka vegetable english centre vege...,Marka : AVC\n\nASHOKA VEGETABLE CENTRE\n\ne EN...,269,887,3863,60,3803,618
1,bill no 2858 militivolves account of punjab gr...,a an rn ee a a Lr AUOLVUIT\n\nBill No. 2858 Da...,253,944,4433,616,3817,691
2,slno,SL.NO.,250,379,1590,1249,341,129
3,item descriptions,“Ast man raroaeonrnrerereex._ “i _\n\nITEM DES...,601,760,2843,1253,1590,159
4,hsn code,HSN CODE,2201,2337,1829,1287,542,136
...,...,...,...,...,...,...,...,...
90,7400,Lee,3615,3693,3004,2589,415,78
91,rrmarks javo 74700 total qty 2190 total amta 2...,Rrmarks\n\nTotal Qty.: 21.90 Total Amt.: 2278.00,226,460,6425,2623,3802,234
92,in words expenses charges 0 00 net payable ter...,in Wor\n\nTerm & Conditions\n\nI/We do hereby ...,174,2879,6638,2798,3840,2705
93,0,,2833,2944,3269,3195,74,111


latest changes

In [40]:
#for info
#dataframes,lineData=getTable(info)
#for i in range(len(lineData)):
#print(lineData[1]['CrnnNetworkOutput'])

#print("-----------------------------")
#df = pd.DataFrame(lineData)

#dataframes.to_csv("info.csv", index=False)

In [14]:
#datapoint=datapoints[1]
inteimResultdf={}
linesdataDict={}
resultData={}

for data in combine:
    ky=list(data.keys())[0]
    print(ky)
    try:
        datapoints=data[ky] 
        interimResults=[]
        resultDF=[]
        linesDataAll=[]
        for datapt in datapoints:
            dataframes,lineData=getTable(datapt)
            interimResults.append(dataframes)
            result=postProcessTable(dataframes)
            resultDF.append(dataframes)
            linesDataAll.append(lineData)
        inteimResultdf[ky]=interimResults
        linesdataDict[ky]=linesDataAll
        resultData[ky]=resultDF
    except Exception as e:
        print(str(e))

CrnnNetworkOutput
exception caught string index out of range m
exception caught string index out of range a
exception caught string index out of range r
exception caught string index out of range k
exception caught string index out of range a
exception caught string index out of range  
exception caught string index out of range a
exception caught string index out of range v
exception caught string index out of range c
exception caught string index out of range  
exception caught string index out of range a
exception caught string index out of range s
exception caught string index out of range h
exception caught string index out of range o
exception caught string index out of range k
exception caught string index out of range a
exception caught string index out of range  
exception caught string index out of range v
exception caught string index out of range e
exception caught string index out of range g
exception caught string index out of range e
exception caught string index out of 

In [15]:
for indx,(key, value) in enumerate(resultData.items()):
  if indx == 2:
    excel_sheet_name = key
    resultDF = value

In [16]:
#resultDF = list(resultData.values())[0]
#excel_sheet_name = list(resultData.keys())[0]
excel_sheet_name = excel_sheet_name[:-4] + 'xlsx'
print(excel_sheet_name)


NameError: ignored

In [ ]:
info = ['GST  Registration  No  :  OSAAACIS950L1ZG', 'SBU  :Packaging  &  Printing  Products  Factory,  Shipment  from  :  ITC  LIMITED', 'Plot  No:1,Sector  -  11,1lE,SIDCUL  Haridwar-249403.  .  SBU  :Packaging  &  Printing  Products  Factory,', 'Plot  No:1,Sector  -  11,E,SIDCUL  Haridwar-249403.', 'y  A zw  Telephone  :91(1334)304701', 'k-nduring  Value  PAN  :  AAACIS950L  :  Telephone  :91(1334)304701', 'Bill  to  (Place  of  Supply)  GST  Invoice  No.  :  H05A218101003578', 'Name :  Tata  Global  Beverages  Ltd.  Name:  Tata  Global  Beverages  Ltd.  Invoice  Date  :  28.02.2018', 'Address  :  PO-Munnar,  Idukki,  Address :  PO-Munnar,  Idukki,  SAP  Doc  No.  :  90280358', 'State  &  State  code:  Kerala  &  32  State  &  State  code:  Kerala  &  32  Document  Currency:  INR', 'GSTIN:  32AABCTO602K1ZU  PAN  :  AABCT0602K  GSTIN:  32AABCT0602K1ZU  PAN:  AABCT0602K', 'HSN  Code}  Part  No./Description  Cust  .  Material  No}  No.  of  |UOM  QTy  Rate  |Net  Value}  Freight  |Pack&  Ing  Taxable  iGST  IGST  Total  TCS  |  Total  Amount', 'Descriptio:', "FLEXIBLE:'{  Tata  Chakra  Gold  Re.1  laminate  -  4500141344", 'Total', 'Total  Invoice  Amount  in  words  :  Rupees  Four  Lakhs  Sixteen  Thousand  Three  Hundred  and  Fifty.  Four  and  Seventy  Six  Paisa  only  (', 'Total  GST  Amount  in  words  Rupees  Sixty  Three  Thousand  Five  Hundred  and  Twelve  only  |  Whether  Tax  is  Payable  on  Reverse  Charge  Basis:  NA', 'Transporter  Name:  RCI  Logistics  Private  Limited  Payment', 'Mode  of  Transportation  :  Ful!  Truck', 'Remarks  :', '_———', '5S=>', 'Vehicle  No  :  UAQ8K  2047', 'LR  number:  397574  LR  Date  :  28.02.2018', "Divisional!  H.Q:  119,St.  Mary's  Road,Abhiramapura", 'm,Chennai  -  600018.  Ph  :(044)  42081506  /  1507  /  1509  Fax  :  (044)  2.', '>  =', ':  Reverse  For  Genereal  Conditions  of  sale)', 'Regd.  Off  :  Virginia  House  37,J.  Nehru  Road,Kolkata  700071, India.  Visit  us  at  www.itcportal.com.  Corporate  Ident  SWB1810PLC601985.  E-mail  :  packexcellence@itc.in']
new_df = pd.DataFrame(info, columns=['Information'])
resultDF.append(new_df)
resultDF

In [38]:
from pandas import ExcelWriter
excel_sheet_name = "ifni_1.xlsx"
writer = ExcelWriter(os.path.join(".", excel_sheet_name))
lineindexsNo=[]
for idx , table in enumerate(lineData):
    #if len(table) > 2 and len(table.columns) > 2 :
        #lineindex=table['lineIndex']
        #lineindexsNo.extend(lineindex)
    table.to_excel(writer, 'sheet' +str(idx), index = False)   
writer.save()